<a href="https://colab.research.google.com/github/soudaminikidambi/PFPgrowth/blob/main/Copy_of_PFP_Growth__Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
!pip install --quiet mrjob==0.7.4

     |████████████████████████████████| 439 kB 5.4 MB/s 


In [ ]:
%pip install mlxtend --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
import csv
import re
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prettytable import PrettyTable

In [ ]:
import mlxtend

mlxtend.__version__

'0.21.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Transactions

In [ ]:
groceries = pd.read_csv("/content/drive/My Drive/datasets/Groceries_dataset.csv")

In [ ]:
all_transactions = [transaction[1]['itemDescription'].tolist() for transaction in list(groceries.groupby(['Member_number', 'Date']))]

In [ ]:
#outFile = "/content/drive/My Drive/datasets/Transactions.csv"
with open("/content/drive/My Drive/datasets/Transactions.csv", 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    writer.writerows(all_transactions)
    

In [ ]:
all_transactions[0:5]

[['sausage', 'whole milk', 'semi-finished bread', 'yogurt'],
 ['whole milk', 'pastry', 'salty snack'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['soda', 'pickled vegetables']]

# Parallel Counting

In [ ]:
%%file parallel_count.py
from mrjob.job import MRJob
import re

class ParallelCount(MRJob):
    def mapper(self, _, line):
      record = line.split(",")
      yield record[2],1
    def reducer(self, item, vals):
      yield item, sum(vals)

if __name__ == '__main__':
     ParallelCount.run()

Writing parallel_count.py


In [ ]:
!python parallel_count.py "/content/drive/My Drive/datasets/Groceries_dataset.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/parallel_count.root.20221217.111808.541206
Running step 1 of 1...
job output is in /tmp/parallel_count.root.20221217.111808.541206/output
Streaming final output from /tmp/parallel_count.root.20221217.111808.541206/output...
"dishes"	135
"dog food"	67
"domestic eggs"	566
"female sanitary products"	40
"finished products"	64
"fish"	29
"flour"	146
"flower (seeds)"	70
"flower soil/fertilizer"	16
"frankfurter"	580
"frozen chicken"	5
"frozen dessert"	92
"frozen fish"	102
"frozen fruits"	11
"frozen meals"	252
"frozen potato products"	72
"frozen vegetables"	425
"fruit/vegetable juice"	518
"grapes"	220
"hair spray"	9
"ham"	256
"hamburger meat"	327
"hard cheese"	220
"herbs"	159
"honey"	13
"house keeping products"	45
"hygiene articles"	208
"ice cream"	227
"instant coffee"	61
"itemDescription"	1
"jam"	34
"ketchup"	32
"kitchen towels"	30
"kitchen utensil"	1
"light bulbs"	29
"lique

# Formation of GLIST

In [ ]:
%%file parallel_count.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

class ParallelCount1(MRJob):
    gr=[]
    def mapper(self, _, line):
      record = line.split(",")
      yield record[2],1
    
    def reducer_init(self):
      self.f = []
    
    def reducer(self,item,vals):
      n = 0
      for i in vals:
        n = n+1
      self.f.append([item,n])
    
    def reducer_final(self):
      yield 1,self.f

    def mapper_init_1(self):
      g=[]
      self.g=g
    
    def mapper_1(self,item,vals):
      for i in vals:
        self.g.append(i[0])
      yield item,self.g

    def reducer_init_1(self):
      flist=[]
      self.flist=flist
    
    def reducer_1(self,item,vals):
      for i in vals:
        self.flist.append(i)
      yield 1,self.flist

    def mapper_init_2(self):
      glist=[]
      self.glist=glist
      Q = 4
      self.Q=Q
    
    def mapper_2(self,item,vals):
      c=0
      a=[]
      #gr=[]
      for i in vals[0]:
        if(c<self.Q):
          a.append(i)
        else:
          if(c%(self.Q)==0):
            self.glist.append([int(c/self.Q), a])
            a=[]
            a.append(i)
          else:
            a.append(i)
        c=c+1
      if(len(a)!=0):
        self.glist.append([int(c/self.Q)+1, a])

      for i in self.glist:
        yield i[0],i
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,reducer_init = self.reducer_init,reducer=self.reducer,reducer_final = self.reducer_final),
            MRStep(mapper_init=self.mapper_init_1,mapper=self.mapper_1,reducer_init = self.reducer_init_1,reducer=self.reducer_1),
            MRStep(mapper_init=self.mapper_init_2,mapper=self.mapper_2),
        ]

if __name__ == '__main__':
    inputFile = "/content/drive/My Drive/datasets/Groceries_dataset.csv"
    outFile = "/content/drive/My Drive/datasets/GLIST.csv"
    mr_job = ParallelCount1(args=[inputFile])
    with mr_job.make_runner() as runner:
      runner.run()
      f = open(outFile, "w") 
      for key, value in mr_job.parse_output(runner.cat_output()):
        s = f'{value}'
        s = s[1:-2]
        s = s.replace('[', '')
        s = s.replace("'", '')
        print(s, file = f)
      f.close()
      print("Done!")


Overwriting parallel_count.py


In [ ]:
!python parallel_count.py 

No configs specified for inline runner
Done!


# Parallel FP

In [ ]:
%%file parallel_count.py
import pandas as pd
import heapq as heap
from mrjob.job import MRJob
from mrjob.step import MRStep
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

tempFile = "/content/drive/My Drive/datasets/GLIST.csv"

class ParallelFP(MRJob):

    def mapper_init(self):
      f = open(tempFile, 'r')
      lines = f.readlines()
      dep = {}
      a=[]
      for line in lines:
        line=line[:-1]
        record = line.split(',')
        a=[]
        for i in range(1,len(record)):
          a.append(record[i])
          a[-1]=a[-1][1:]
        dep[record[0]]=a
      self.dep = dep

    def mapper(self, _, line):
      record = line.split(",")
      dep=self.dep
      for x in range(len(record)):
        a=[]
        for i in dep:
          if record[x] in dep[i]:
            a.append(record[x])
            for j in range(x+1,len(record)):
              if record[j] in dep[i]:
                a.append(record[j])
            yield i,a

    def reducer(self,item,vals):
      x=[]
      for i in vals:
        x.append(i)
      yield item,x

    def reducer_1(self,item,vals):
      for i in vals:
        trans_encoder = TransactionEncoder() # Instanciate the encoder
        trans_encoder_matrix = trans_encoder.fit(i).transform(i)
        trans_encoder_matrix = pd.DataFrame(trans_encoder_matrix, columns=trans_encoder.columns_)
        res=fpgrowth(trans_encoder_matrix,min_support=0.0000001, use_colnames=True)
        support = []
        item_set = []
        item_dict = {}
        for i in res.index:
          support.append(float(res['support'][i]))
          item_set.append(list(res['itemsets'][i]))
          item_dict[support[-1]]=item_set[-1]
        yield item, item_dict      

    def mapper_init_2(self):
      self.l=[]

    def mapper_2(self,item,vals):
      for i in vals:
        heap.heappush(self.l,(float(i),vals[i]))
        if(len(self.l)>20):
          heap.heappop(self.l)
    
    def mapper_final_2(self):
      for i in self.l:
        yield None, i

    def reducer_init_2(self):
        self.l=[]

    def reducer_2(self,item,vals):
      l1=self.l
      for i in vals:
        heap.heappush(l1,i)
        if(len(l1)>20):
          heap.heappop(l1)
    
    def reducer_final_2(self):
      self.l.sort(reverse=True)
      for i in self.l:
        yield  None,[i[0], i[1]]

    def mapper_init_3(self):
      self.l=[]

    def mapper_3(self,item,vals):
      heap.heappush(self.l,(float(vals[0]),vals[1]))
      if(len(self.l)>20 or self.l[0][0] < float(vals[0])):
        heap.heappop(self.l)
    
    def reducer_3(self):
      self.l.sort(reverse=True)
      yield  None, self.l

    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,mapper=self.mapper,reducer=self.reducer),
            MRStep(reducer=self.reducer_1),
            MRStep(mapper_init=self.mapper_init_2,mapper = self.mapper_2,mapper_final = self.mapper_final_2,reducer_init=self.reducer_init_2,reducer = self.reducer_2,reducer_final=self.reducer_final_2)
            #MRStep(mapper_init=self.mapper_init_3,mapper = self.mapper_3, reducer = self.reducer_3)
        ]

if __name__ == '__main__':
    ParallelFP.run()


Overwriting parallel_count.py


In [ ]:
!python parallel_count.py "/content/drive/My Drive/datasets/Transactions.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/parallel_count.root.20221217.153015.121905
Running step 1 of 3...
Running step 2 of 3...
Running step 3 of 3...
job output is in /tmp/parallel_count.root.20221217.153015.121905/output
Streaming final output from /tmp/parallel_count.root.20221217.153015.121905/output...
null	[1.0, ["other vegetables"]]
null	[0.836940836940837, ["frankfurter"]]
null	[0.8075657894736842, ["margarine"]]
null	[0.7079207920792079, ["domestic eggs"]]
null	[0.6115618661257607, ["newspapers"]]
null	[0.5766590389016019, ["frozen meals"]]
null	[0.558282208588957, ["long life bakery product"]]
null	[0.5462633451957295, ["onions"]]
null	[0.48031496062992124, ["instant coffee"]]
null	[0.47572815533980584, ["flour"]]
null	[0.46247464503042596, ["ice cream"]]
null	[0.44880546075085326, ["fruit/vegetable juice"]]
null	[0.43478260869565216, ["kitchen towels"]]
null	[0.4279918864097363, ["hygiene artic